In [1]:
import pandas as pd
import numpy as np

In [2]:
weather_total = pd.read_csv('final_weather_table.txt',low_memory=False)
weather_total.head()

,Unnamed: 0,YYMM,State,WBAN,Latitude,Longitude,AvgTemp,TotalMonthlyPrecip,Avgwindspeed
0,0,199607,AK,25704,51.883,-176.650,51.6,0.22,NaN
1,1,199607,AL,13894,30.688,-88.246,81.7,4.95,5.6
2,2,199607,AL,13895,32.301,-86.394,80.8,8.72,4.8
3,3,199607,AL,3856,34.644,-86.786,78.2,4.7,5.6
4,4,199607,AR,13964,35.334,-94.365,79.7,7.67,5.3


In [3]:
temp = weather_total.iloc[:,[1,2,3,-3]]
temp.replace('M',np.nan,inplace=True)
temp.dropna(inplace=True)

C:\Users\Jiayuan\Miniconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Jiayuan\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [85]:
#revise the type of columns
temp['YYMM'] = temp['YYMM'].astype(str)
temp['AvgTemp'] = temp['AvgTemp'].astype(float)

C:\Users\Jiayuan\Miniconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\Jiayuan\Miniconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [88]:
#run all the year of AZ
tempsw = pd.DataFrame()
for j in range(1997,2017):
    
    year = AZ[AZ['YYMM'].str.contains('j')]

In [75]:
#1997, AZ
#for loop to get month list
AZ_1997 = AZ[AZ['YYMM'].str.contains("1997")]
YYMM_list = list(AZ_1997['YYMM'])#含有重复月份的list
#获得不含重复月份的list 
month_index = []
for i in YYMM_list:
    if not i in month_index:
        month_index.append(i)

# run all the month in 1997 and get the dataframe（named avg_temp） of AZ in 1997
avg = []
avg_temp = pd.DataFrame()
for i in month_index:
    month_df = AZ[AZ['YYMM'].isin([i])]
    avg.append(sum(list(month_df['AvgTemp']))/len(month_df))
    avg_temp['YYMM'] = month_index
avg_temp['AvgTemp'] = avg
print(avg_temp)

      YYMM    AvgTemp
0   199701  43.442857
1   199702  45.014286
2   199703  55.314286
3   199704  57.257143
4   199705  71.485714
5   199706  74.250000
6   199707  78.128571
7   199708  77.828571
8   199709  73.433333
9   199710  56.771429
10  199711  48.828571
11  199712  37.571429


In [77]:
#Find the tempsummer and tempwinter in a specific year for a state
tmax_index = avg_temp['AvgTemp'].idxmax()
tempmax_three = 0
for i in range(tmax_index-1,tmax_index+2):
    tempmax_three += avg_temp.iloc[i,-1]
tempsummer = tempmax_three/3

tmin_index = avg_temp['AvgTemp'].idxmin()
tempmin_three = 0
for i in range(tmin_index-1,tmin_index+2):
    if i>11:
        i = i-12# index jump back to the first month
    tempmin_three += avg_temp.iloc[i,-1]
tempwinter = tempmin_three/3

In [ ]:
#For loop for all 51 states
statelist = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 
             'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 
             'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 
             'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [81]:
#extract one state and reset index
#for AZ in 1996
AZ = temp[temp['State'].isin(['AZ'])]
AZ_199607 = AZ[AZ['YYMM'].isin(['199607'])]
#avg_AZ_199607 = sum(list(AZ_199607['AvgTemp']))/len(list(AZ_199607['AvgTemp']))
#sum(list(AZ_199607['AvgTemp']))
#avg_AZ_199607
